In [ ]:
#Finding mutant in TLH gene (ORF) & Finding transversion and transition and counting them
# writting by Rahnama Lab
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import AlignInfo
from Bio import AlignIO
format_ = "fasta"
filename = "test.fasta" #Change with your Sequence fasta format
alignment = AlignIO.read(filename, format_)
msa_summary = AlignInfo.SummaryInfo(alignment)
cons=  msa_summary.dumb_consensus(0.5)

In [ ]:
for rec in alignment:
    print(rec.id)

In [ ]:
corr_align={}
rec_dic={}
for record in alignment:
    rec_dic[record.id]=str(record.seq).upper()
    seq=list(str(record.seq).upper())

    for z,_ in enumerate(seq):
            if seq[z] == "-":
                seq[z]="N"
            else:
                break
    seq=''.join(seq)
    corr_align[record.id]=seq
    print("%s %s" % (record.id, seq))
records =[ SeqRecord(
                          Seq(corr_align[i]),
                          id=i,
                      ) for i in corr_align]
SeqIO.write(records, "tmp.fa", "fasta")

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import AlignInfo
from Bio import AlignIO
format_ = "fasta"
filename = "tmp.fa"
alignment = AlignIO.read(filename, format_)
msa_summary = AlignInfo.SummaryInfo(alignment)
cons=  msa_summary.dumb_consensus(0.5)

In [ ]:
corr_align={}
rec_dic={}
for record in alignment:
    rec_dic[record.id]=str(record.seq).upper()
    seq=list(str(record.seq).upper())
    for z,i in enumerate(seq):
        if seq[z] == "N":
            seq[z]="-"
    seq=''.join(seq)
    corr_align[record.id]=seq
    print("%s %s" % (record.id, seq))

In [ ]:
records =[ SeqRecord(
                          Seq(corr_align[i]),
                          id=i,
                      ) for i in corr_align]
SeqIO.write(records, "cons.fa", "fasta")

In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import AlignInfo
from Bio import AlignIO
format_ = "fasta"
filename = "cons.fa"
alignment = AlignIO.read(filename, format_)
msa_summary = AlignInfo.SummaryInfo(alignment)
cons=  msa_summary.dumb_consensus(0.5)

In [ ]:
align_seqs={}
for record in alignment:
    align_seqs[record.id]=str(record.seq).upper()

In [ ]:
cons=str(cons)
cns_list=list(cons)[:]
table={}
for z,inf in enumerate(cns_list):
    data={"consensus":inf}
    cond=False
    indicator=sum([align_seqs[seq_id][z]==inf for seq_id in align_seqs])/len(align_seqs)
    for seq_id in align_seqs:
     #   print(align_seqs[seq_id][z],inf)
        if align_seqs[seq_id][z]==inf:
            if indicator<0.5:
                data[seq_id]=inf
                cond = True
            else:
                data[seq_id]="."
                cond=len(set([data[i] for i in data]))>2
        else:
            data[seq_id]=align_seqs[seq_id][z]
            
    if indicator<0.5:
        data["consensus"]="-"
    seq_list=[rec_dic[s][z] for s in align_seqs]
    condition2=sum([data[i]=="-" and seq_list[z-1]!="N" for z,i in enumerate(data) if i!="consensus"])

    if cond and ((set([data[i] for i in data if i!="consensus"])!={"-","."}) or (condition2<0.5*len(align_seqs) and condition2>0)) :
        table[z+1]=data

In [ ]:
import pandas as pd

In [ ]:
df=pd.DataFrame(table)
#df.columns=cns_list
#df.to_cs
df.to_csv("seq_table.csv")

In [ ]:
cols=list(df.columns)
count={}
for col in cols:
    cls={}
    cons=df[col]["consensus"]
    for i in df[col]:
        if i!=cons and i!=".":
            mut=cons+">"+i
            if cls.get(mut,None) is None:
                cls[mut]=1
            else:
                cls[mut]+=1
    count[col]=cls

In [ ]:
cnt_df=pd.DataFrame(count).fillna(0)

In [ ]:
cnt_df["sum"]=cnt_df.sum(axis=1)

In [ ]:
cnt_df=cnt_df[["sum"]+list(cnt_df.columns)[1:]]
cnt_df.to_csv("counts.csv")

In [ ]:
m=msa_summary.pos_specific_score_matrix().pssm
pssm=[[i[0]] +[i[1][l] for l in  i[1]]  for i in m]
pssm_df=pd.DataFrame(pssm).T
pssm_df.index=["consensus","A","C","G","T"]
pssm_df.columns=[i+1 for i in range(len(cns_list))] 

In [ ]:

vals=[pssm[i-1] for i in table.keys()]
vals=[i for i in vals if sum([j!=0 for j in i[1:]])>1]

In [ ]:

vals=[pssm[i-1] for i in table.keys()]
vals=[i for i in vals if sum([j!=0 for j in i[1:]])>1]
pssm_df=pd.DataFrame(vals).T
pssm_df.index=["consensus","A","C","G","T"]
pssm_df.to_csv("pssm.csv")

In [ ]:
dic={
    "A>T":0,"A>C":0,"A>G":0,
    "T>A":0,"T>C":0,"T>G":0,
    "C>A":0,"C>T":0,"C>G":0,
    "G>A":0,"G>T":0,"G>C":0,
    "X>A":0,"X>T":0,"X>C":0,"X>G":0,
    "A>X":0,"T>X":0,"C>X":0,"G>X":0
    }

In [ ]:
chrs=["A","C","G","T"]
for v in vals:
    for z,i in enumerate(v[1:]):
        if i>0 and v[0]!=chrs[z]:
            dic[v[0]+">"+chrs[z]]=dic[v[0]+">"+chrs[z]]+1
pd.DataFrame([(i,dic[i]) for i in dic],columns=["mutation","count"]).to_csv("count.csv",index=False)